In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.constants import G, c
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

### Step 1: Importing Necessary Libraries
# Constants
H_0 = cosmo.H(0).to(u.m/u.s/u.Mpc).value  # Hubble constant in SI units (m/s/Mpc)
c_val = c.to(u.m/u.s).value  # Speed of light in m/s
G_val = G.to(u.m**3/u.kg/u.s**2).value  # Gravitational constant in SI units
q0 = -0.534

### Step 2: Reading the CSV Data
df = pd.read_csv("/mnt/data/Skyserver_6_17_2025 1_03_52 AM.csv")

### Step 3: Calculating the Average Spectroscopic Redshift (specz) for Each Object
averaged_df = df.groupby('objid').agg({'specz': 'mean','ra': 'first','dec': 'first','proj_sep': 'first',}).reset_index()
averaged_df.describe()['specz']

### Step 4: Redshift Cut to Identify Cluster Members
mean_z = averaged_df['specz'].mean()
std_z = averaged_df['specz'].std()
lower_limit = mean_z - 3 * std_z
upper_limit = mean_z + 3 * std_z

plt.figure(figsize=(8, 5))
plt.hist(averaged_df['specz'], bins=50, color='skyblue', edgecolor='black')
plt.title("Distribution of Redshift for the Field")
plt.xlabel("Spectroscopic Redshift (specz)")
plt.ylabel("Number of Galaxies")
plt.grid()
plt.show()

filtered_df = averaged_df[(averaged_df['specz'] > lower_limit) & (averaged_df['specz'] < upper_limit)].copy()

### Step 5: Adding Velocity Column
filtered_df['velocity'] = filtered_df['specz'] * c_val / (1 + filtered_df['specz'])

plt.figure(figsize=(8, 5))
plt.hist(filtered_df['velocity'], bins=40, color='salmon', edgecolor='black')
plt.title("Velocity Distribution of Cluster Members")
plt.xlabel("Velocity (km/s)")
plt.ylabel("Number of Galaxies")
plt.grid()
plt.show()

### Step 6: Calculating Velocity Dispersion
disp = filtered_df['velocity'].std()

### Step 7: Calculating Mean Redshift of the Cluster
cluster_redshift = filtered_df['specz'].mean()

print(f"Cluster Redshift = {cluster_redshift:.4f}")
print(f"Velocity Dispersion = {disp/1000:.2f} km/s")

### Step 8: Visualizing Angular Separation of Galaxies
plt.figure(figsize=(8, 5))
plt.hist(filtered_df['proj_sep'], bins=40, color='lightgreen', edgecolor='black')
plt.title("Projected Angular Separation")
plt.xlabel("Projected Separation (arcmin)")
plt.ylabel("Number of Galaxies")
plt.grid()
plt.show()

### Step 9: Estimating Physical Diameter of the Cluster
r_comoving = (c_val * cluster_redshift / H_0) * (1 - (cluster_redshift / 2) * (1 + q0))
D_A = r_comoving / (1 + cluster_redshift)

theta_rad = np.radians(filtered_df['proj_sep'].max() / 60)  # arcmin -> deg -> rad
diameter = D_A * theta_rad
print(f"Cluster Physical Diameter = {diameter:.2f} Mpc")

### Step 10: Calculating the Dynamical Mass of the Cluster
R_m = (diameter / 2) * 3.086e22  # Mpc -> meters
sigma_m_s = disp  # already in m/s
M_dyn_kg = 3 * sigma_m_s**2 * R_m / G_val
M_dyn_solar = M_dyn_kg / (1.989e30)

print(f"Dynamical Mass of the Cluster = {M_dyn_solar:.2e} solar masses")

### Summary (Answers to Questions from PDF)
# (1) Identifying cluster members:
#    - Using 3-sigma clipping on the redshift distribution to select galaxies close in redshift.
# (2) Cluster Redshift and Velocity Dispersion:
#    - Cluster redshift = {cluster_redshift:.4f}
#    - Velocity dispersion = {disp/1000:.2f} km/s
# (3) Characteristic Size:
#    - Diameter of the cluster ≈ {diameter:.2f} Mpc
# (4) Dynamical Mass:
#    - Dynamical mass ≈ {M_dyn_solar:.2e} solar masses
# (5) Consistency with Luminous Mass:
#    - Typical luminous mass for rich clusters is ~10^14 to 10^15 solar masses.
#    - Our estimated dynamical mass is within this expected range.
#    - If there is a discrepancy, it may be due to:
#        - galaxies not in equilibrium,
#        - projection effects,
#        - interlopers,
#        - underestimated or overestimated velocity dispersion.
#
#    - In this case, dynamical mass is roughly consistent with luminous mass estimates for typical clusters.

# Plots explored:
# - Histogram of redshift
# - Histogram of velocity
# - Histogram of projected separation
